# Introduction
<p> The main objective of this file is to manipulate the data according to the needs of our stakeholders. </p>

Required Imports

In [1]:
# Data Imports
import pandas as pd
import numpy as np
import json

# MySQL Imports
import pymysql
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists

In [2]:
# Get SQL creds from json
creds = json.load(open('creds.json'))

pymysql.install_as_MySQLdb()

# Create connection string using credentials following this format
database = "imdb"
ip_addr = 'localhost'
connection_str = f"mysql+pymysql://{creds['username']}:{creds['password']}@{ip_addr}/{database}"
    
# Create database if it doesn't exist
if database_exists(connection_str) == False: 
    create_database(connection_str)
else: 
    print(f'Database {database} exists.')

# Create Engine
engine = create_engine(connection_str)

Database imdb exists.


# Title Basics Data
<p> Configure Title Basics Data to our needs </p>

Let's load the data and view it to get an idea

In [3]:
basics = pd.read_csv('Data/title.basics.tsv.gz', sep = '\t', low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


We need to manipulate the data so it is as follow:<br>

1. Replace "\N" with np.nan<br>
2. Eliminate movies that are null for runtime<br>
3. Eliminate movies that are null for genre<br>
4. Keep only titleType == Movies<br>
5. Keep only between startYear 2000-2022 (excluding 2022)<br>
6. Eliminate movies that include  "Documentary" in genre

In [4]:
# fix null values
basics = basics.replace({'\\N': np.nan})

# remove runtimes if they aren't a number
# found runtimeMinutes has genres in it - get rid of em
drops = []
for var in [x for x in basics['runtimeMinutes'] if type(x) == type('')]:
    try:
        # if we can typecast to int then it was wrong format
        int(var)
    except:
        # if we can't typecast to int then it was a string
        if var not in drops:
            drops.append(var)
# drop columns where runtime equals str vals
for val in drops:                
    basics = basics.drop(basics.loc[basics['runtimeMinutes'] == val].index)

# drop empty values from these columns
basics = basics.dropna(subset = ['runtimeMinutes', 'genres', 'startYear'])
# Only Movies
basics = basics.loc[basics['titleType'] == "movie"]
# No Documentaries
basics = basics[~basics['genres'].str.contains("Documentary", case = False)]
# Only between 2000 and 2021
basics = basics.loc[(basics['startYear'].astype(int) >= 2000) & (basics['startYear'].astype(int) <= 2021)]

# Title Ratings Data 
<p> Configure Title Ratings Data to fit our needs </p>

Loand and view the data

In [5]:
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep = '\t', low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1876
1,tt0000002,5.9,248
2,tt0000003,6.5,1649
3,tt0000004,5.8,160
4,tt0000005,6.2,2475


Manipulate then save data

In [6]:
# fix null values
ratings = ratings.replace({'\\N': np.nan})

# save data
ratings.to_csv("Data/ratings.csv.gz", compression='gzip', index=False)

# Title Akas Data
<p> Configure Title Akas Data to fit our needs </p>

Load and view the data

In [7]:
akas = pd.read_csv('Data/title.akas.tsv.gz', sep = '\t', low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [8]:
# fix null values
akas = akas.replace({'\\N': np.nan})
# US only movies
akas = akas.loc[akas['region'] == 'US']

Now we must filter the basics data to US only movies and save them both

In [9]:
# Filter the basics table down to only include the US by using the filter akas dataframe
basics = basics[basics['tconst'].isin(akas['titleId'])]

# save data
akas.to_csv("Data/akas.csv.gz", compression='gzip', index=False)
basics.to_csv("Data/basics.csv.gz", compression='gzip', index=False)